## Converting data from netCDF4 to IMMA1 format

Developed by Senya Stein (senyarocks11@gmail.com) in July 2019 for use with Saildrone data

IMMA documentation is at https://rda.ucar.edu/datasets/ds548.0/#!docs

Can be used with conversion from IMMA code developed by Zhankun Wang and Phillip Brohan: https://github.com/oldweather/IMMA/blob/master/Python/IMMA/icoads.py

In [2]:
import numpy as np
import pandas as pn
import xarray as xr
import matplotlib.pyplot as plt
import datetime as dt
import netCDF4

Note: IMMA expects specific units and instrumentation, this program assumes the following incoming data, instruments, and country codes:

Latitude-Degrees North (to hundredths+)

Longitude-Degrees East (to hundredths+)

Time-Hours plus minutes

Country that recruited ship- United States (C1_var, will need manual change)

Wind direction compass precision- Tenths+ of degrees

Wind speed units- knots, obtained from anemometer (measured)

Sea level pressure- Hpa

Sea surface temperature- taken with a through hull sensor ASK IF CORRECT TO IMMA PPL (pumped CTD measurment)

Wave period and wave height- don't exist at default but easily added (line 60)




### Data in xarray form

In [3]:
url = 'https://podaac-opendap.jpl.nasa.gov/opendap/hyrax/allData/insitu/L2/saildrone/Baja/saildrone-gen_4-baja_2018-sd1002-20180411T180000-20180611T055959-1_minutes-v1.nc'
ds = xr.open_dataset(url, drop_variables = {'WING_ANGLE','BARO_PRES_STDDEV', 'ROLL', 'PITCH', 'TEMP_AIR_STDDEV', 'RH_STDDEV', 'UWND_STDDEV', 'VWND_STDDEV', 'GUST_WND_STDDEV', 'TEMP_CTD_STDDEV', 'COND_STDDEV', 'SAL_STDDEV', 'O2_CONC_UNCOR_MEAN', 'O2_CONC_UNCOR_STDDEV', 'O2_SAT_MEAN', 'O2_SAT_STDDEV', 'TEMP_O2_MEAN', 'TEMP_O2_STDDEV', 'CHLOR_MEAN', 'CHLOR_STDDEV', 'BKSCT_RED_MEAN', 'BKSCT_RED_STDDEV', 'CDOM_STDDEV', ' WWND_STDDEV', 'TEMP_IR_UNCOR_STDDEV' })
ds

<xarray.Dataset>
Dimensions:             (obs: 86839, trajectory: 1)
Coordinates:
  * trajectory          (trajectory) float32 1002.0
    time                (trajectory, obs) datetime64[ns] ...
    latitude            (trajectory, obs) float64 ...
    longitude           (trajectory, obs) float64 ...
Dimensions without coordinates: obs
Data variables:
    SOG                 (trajectory, obs) float64 ...
    COG                 (trajectory, obs) float64 ...
    HDG                 (trajectory, obs) float64 ...
    HDG_WING            (trajectory, obs) float64 ...
    BARO_PRES_MEAN      (trajectory, obs) float64 ...
    TEMP_AIR_MEAN       (trajectory, obs) float64 ...
    RH_MEAN             (trajectory, obs) float64 ...
    TEMP_IR_UNCOR_MEAN  (trajectory, obs) float64 ...
    UWND_MEAN           (trajectory, obs) float64 ...
    VWND_MEAN           (trajectory, obs) float64 ...
    WWND_MEAN           (trajectory, obs) float64 ...
    WWND_STDDEV         (trajectory, obs) float64 .

In [6]:
print(ds.TEMP_AIR_MEAN[0])

<xarray.DataArray 'TEMP_AIR_MEAN' (obs: 86839)>
array([12.99, 13.12, 13.08, ...,   nan,   nan,   nan])
Coordinates:
    trajectory  float32 1002.0
    time        (obs) datetime64[ns] ...
    latitude    (obs) float64 ...
    longitude   (obs) float64 ...
Dimensions without coordinates: obs
Attributes:
    standard_name:              air_temperature
    long_name:                  Air temperature
    units:                      degrees_c
    update_period:              Attribute edlided: Unsupported attribute type...
    model_product_page:         
    device_name:                Rotronic AT/RH (1053)
    installed_date:             2017-10-02T18:47:07Z
    nominal_sampling_schedule:  60s on, 240s off, centered at :00
    installed_height:           2.4
    serial_number:              1053
    vendor_name:                Rotronic
    model_name:                 HC2-S3


In [47]:
# swap obs for time
ds=ds.isel(trajectory=0)
ds = ds.swap_dims({'obs':'time'})


In [48]:
#resample to make time = 1 hr increments

pd_ds = ds.to_dataframe()
dshr = pd_ds.set_index('time').groupby(pd.Grouper(freq='1H')).mean()
dshr = ds
dshr=ds.resample(time='1h', skipna=True, label='left').mean()
dshr
# does data need to be averaged from beginning of hour to end, or from half way through one hour and half through the other??
#for now, we will average hours from beginning of hour to the end, how resample defaults
#data showed as 18:00 is data from the hour of 18:00 to 18:59
 
#size = ds['time'].to_data_array()
size = int(len(ds['time']))
print(size)

#size = ds.sizes('time')

86839


# Do the math stuff in xarray
Then save as a dataframe, then to strings of the data 

In [8]:
time_shift = dshr.time + np.timedelta64(30,'m')

for i in range(size):
   # HR = str(int(100*(time_shift.dt.hour[i]+time_shift.dt.minute[i]/60)/100))
    #format string output and put hours in correct units (.01hr)
    HR = '{time_shift.dt.hour[i]+time_shift.dt.minute[i]/60:.2f}'
    HR = str(HR)
           
    time_str = str(time_shift.dt.year[i])+str(time_shift.dt.month[i])+str(time_shift.dt.day[i])+HR
    #time_str is YR + MO + DY + HR
    

#format output to 2 decimal places
#ds_hour = str(dshr['time.hour'])
#precision = 2
#ds_hour = '{ds_hour:.2f}'

# do the same for lat and lon 
lat_shift = ds.latitude
lon_shift = ds.longitude

# do lat and lon need shifted units or not?
for i in range(size): 
    LAT = '{lat_shift[i]:.2f}'
    LON = '{lon_shift[i]:.2f}'
    pos_str = str(LAT[i])+str(LON[i])

#ds_hour = dshr.resample(time = '.01H', skipna = True, label = 'left')
#put it in .01 hour for IMMA, need to do the same for LAT and LON

#ds_day = dshr['time.day']
#ds_day 

#ds_month = dshr['time.month']
#ds_month

#ds_year = dshr['time.year']
#ds_year

NameError: name 'dshr' is not defined

In [ ]:
for i in range(size):
    IM = '01' 

#ATTC-1 p.19

ATTC = ' '
#bc saildrone doesnt have the above
#rename blank strings to something more intuitive later?


TI_var = '2'
LI_var = '5'

#COG and SOG now
DS_var = ds.COG
DV_var = ds.SOG
    
for i in range(size):
    DSDV = str(DS_var[i])+str(DV_var[i])
    
#NID-2 
NID = '  '


II = '  '#2
ID = '         '#9
C1 = '02'
### C1 is The country that recruited a ship, which may differ from the country of immediate receipt, see page 21, # 16 in IMMA documentation for full list of codes


In [12]:
for i in range(size):
    DI_var = '6'
    D_var = np.arctan2(ds.VWND_MEAN[i], ds.UWND_MEAN[i])*180/np.pi #this is wind TO direction, i think imma format is wind FROM
    WI_var = '4'
    W_var = np.sqrt(ds.UWND_MEAN[i]**2 + ds.VWND_MEAN[i]**2) #convert vectors to speed
    W_var = W_var/1.9444  #unit conversion (kn to .1 m/s)
    W_var = '{W_var:.1f}'


#is wind speed given at 10m height for IMMA?  -- it doesnt say in pdf -- if so, you need to convert to 10m wind as follows:
#WS_height=int(ds.UWND_MEAN.installed_height)
#wind_speed_10m = (wind_speed*log(10./1e-4))/log(WS_height/1e-4)


#VI-1 VV-2 WW-2 W1- 1
VItoW1 = '      '


for i in range(size):
#reformat the SLP and put in correcnt units
    SLP_var = ds.BARO_PRES_MEAN[i]
    SLP_var = SLP_var[i]*.1
    SLP_var = '{SLP_var:.1f}'

#A-1 PPP-3 IT-1
AtoIT = '    '


for i in range(size):
    IT_var = '8'

    AT_var = ds.TEMP_AIR_MEAN[i]
    AT_var = AT_var*.1
    AT_var = '{AT_var:.1f}'

#WBTI-1 WBT-4 DPTI-1 DPT-4 
WBTItoDPT = '          '

SI_var = '04'
    
for i in range(size):
    SST_var = ds.TEMP_CTD_MEAN[i]
    SST_var = SST_var*.1
    SST_var = '{SST_var:.1f}'

#N-1 NH-1 CL-1 HI-1 H-1 CM-1 CH-1 
NtoCH = '       '

if WAVE_DOMINANT_PERIOD != nan:
    for i in range(size):
        WD_var = '  '
        WP_var = WAVE_DOMINANT_PERIOD
        WH_var = WAVE_SIGNIFICANT_HEIGHT
else:
    #WD-2 WP-2 WH-2
    WD_var = '  '
    WP_var = '  '
    WH_var = '  '


#everything beyond that I dont think we use (the lengths of them):
##DOUBLE CHECK THIS
#1 3 2 3 3 2 2 1 1 1 1 1 2 12 1 6 14 2 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 2 2 2 1 1 1 1 1 1 1 2 2 2 1 2 1 1 1 1 1 1 1 3 1 1 1 20 1 3 3 2 2 3 3 3 8 4 1 1 7 2 2 4 6 1 5 4 4 4 4 4 3 3 5 1 4 4 2 2 2 2 2 2 1 2 2 2 2 3 3 2 2 1 2 3 3 2 3 3 3 3 5 5 2 2 2 5 4 5 4 4 4 4 4 5 4 5 4 3 4 4 4 3 4 4 4 2 4 10 2 2 1 2 1 1 1 2 2 1 3 3 1 1 1 4 4 2 2 2 2 2 2 1 7 7 7 7 7 4 8 1 2 2 2 2 1 6 1 1 6 1 1 6 1 1 6 1 4 8 1 2 2 2 2 1 10 4 8 1 2 2 6 1 1 1 1 1 2 2 1 
#1+3+2+3+3+2+2+1+1+1+1+1+2+12+1+6+14+2+1+2+1+1+1+1+1+1+1+1+1+1+1+1+1+1+1+1+1+1+1+1+1+2+1+2+2+2+1+1+1+1+1+1+1+2+2+2+1+2+1+1+1+1+1+1+1+3+1+1+1+20+1+3+3+2+2+3+3+3+8+4+1+1+7+2+2+4+6+1+5+4+4+4+4+4+3+3+5+1+4+4+2+2+2+2+2+2+1+2+2+2+2+3+3+2+2+1+2+3+3+2+3+3+3+3+5+5+2+2+2+5+4+5+4+4+4+4+4+5+4+5+4+3+4+4+4+3+4+4+4+2+4+10+2+2+1+2+1+1+1+2+2+1+3+3+1+1+1+4+4+2+2+2+2+2+2+1+7+7+7+7+7+4+8+1+2+2+2+2+1+6+1+1+6+1+1+6+1+1+6+1+4+8+1+2+2+2+2+1+10+4+8+1+2+2+6+1+1+1+1+1+2+2+1
#607+4+6

ALL_var = ''
for i in range(617):
       ALL_var += ' '

NameError: name 'size' is not defined

NameError: name 'time_str' is not defined

### Decode and store data as Pandas

In [ ]:
pd_ds = ds.to_dataframe()
pd_ds

In [ ]:
pd_ds[['time', 'PITCH']]

In [ ]:
#write dataframe to file
pd_ds.to_csv("test1.csv")  
#read it back
pd.read_csv("test1.csv").head()

### Convert new csv file to a text file

In [ ]:
# A simple program to create a formatted text file from a *.csv file.

csv_file = input('test1.csv')
txt_file = input('test1.txt')

try:
    my_input_file = open(csv_file, "r")
except IOError as e:
    print("I/O error({0}): {1}".format(e.errno, e.strerror))

if not my_input_file.closed:
    text_list = [];
    for line in my_input_file.readlines():
        line = line.split(",", 2)
        text_list.append(" ".join(line))
    my_input_file.close()

try:
    my_output_file = open(txt_file, "w")
except IOError as e:
    print("I/O error({0}): {1}".format(e.errno, e.strerror))

if not my_output_file.closed:
    my_output_file.write("#1\n")
    my_output_file.write("double({},{})\n".format(len(text_list), 2))
    for line in text_list:
        my_output_file.write("  " + line)
    print('File Successfully written.')
    my_output_file.close()